# Blueprint System Tutorial

The **Blueprint System** allows you to define agents in YAML and generate Python code automatically.

## Overview

- **Type**: Meta-Agent (generates other agents)
- **Use Case**: Define agents declaratively, generate consistent implementations
- **Features**: Schema validation, code generation, two generation modes

## How Blueprints Work

```
┌─────────────────────┐     ┌─────────────────────┐     ┌───────────────────┐
│  YAML Blueprint     │ --> │   AgentBuilder      │ --> │  Python Agent     │
│  (specification)    │     │   (meta-agent)      │     │  (generated code) │
└─────────────────────┘     └─────────────────────┘     └───────────────────┘
```

1. **Define** your agent in a YAML blueprint file
2. **Validate** the blueprint structure and semantics
3. **Generate** Python code using templates
4. **Write** the code to your agents directory

## Setup

In [ ]:
from agent_workshop.blueprints import (
    AgentBuilder,
    load_blueprint,
    validate_blueprint,
)
from agent_workshop import Config

config = Config()
builder = AgentBuilder(config)

print(f"Provider: {builder.provider_name}")
print(f"Model: {builder.model_name}")

## Blueprint Structure

A blueprint YAML file has two main sections:

```yaml
# blueprints/specs/my_agent.yaml

blueprint:
  version: "1.0"
  name: "my_agent"
  domain: "my_domain"  # e.g., software_dev, data_science
  description: "What the agent does"
  type: "simple"  # or "langgraph" for workflows

agent:
  class_name: "MyAgent"
  
  input:
    type: "string"  # or "dict"
    description: "What the agent expects"
    
  output:
    type: "dict"
    schema:
      result: "str"
      score: "int"
      
  prompts:
    system_prompt: |
      System prompt text...
    user_prompt_template: |
      User prompt with {placeholders}...
      
  validation_criteria:
    - "First criterion"
    - "Second criterion"
```

## Loading and Validating Blueprints

In [ ]:
# Load an existing blueprint
blueprint_path = "blueprints/specs/software_dev_code_reviewer.yaml"

try:
    blueprint = load_blueprint(blueprint_path)
    print("Blueprint loaded successfully!")
    print(f"  Name: {blueprint.blueprint.name}")
    print(f"  Domain: {blueprint.blueprint.domain}")
    print(f"  Type: {blueprint.blueprint.type}")
    print(f"  Class: {blueprint.agent.class_name}")
except FileNotFoundError:
    print(f"Blueprint file not found: {blueprint_path}")
    print("Make sure you're running from the project root.")

In [ ]:
# Validate blueprint semantics
if 'blueprint' in dir():
    validation = validate_blueprint(blueprint)
    
    print("Validation Result:")
    print(f"  Valid: {validation.valid}")
    
    if validation.errors:
        print("  Errors:")
        for error in validation.errors:
            print(f"    - {error}")
    
    if validation.warnings:
        print("  Warnings:")
        for warning in validation.warnings:
            print(f"    - {warning}")

## Generating Agent Code

In [ ]:
# Generate code without writing to file
# result = await builder.run({
#     "blueprint_path": "blueprints/specs/software_dev_code_reviewer.yaml",
# })
#
# if result["success"]:
#     print("Generated Code:")
#     print("=" * 50)
#     print(result["code"][:1000] + "...")  # First 1000 chars
# else:
#     print(f"Generation failed: {result['error']}")

print("Run the cell above (uncomment) to generate code")

In [ ]:
# Generate and write to file
# result = await builder.run({
#     "blueprint_path": "blueprints/specs/software_dev_code_reviewer.yaml",
#     "output_path": "src/agent_workshop/agents/my_code_reviewer.py",
#     "overwrite": True,  # Required if file exists
# })
#
# if result["success"]:
#     print(f"Agent written to: {result['written_path']}")
# else:
#     print(f"Failed: {result['error']}")

print("Run the cell above (uncomment) to generate and write code")

## Convenience Function

For simpler usage, use the `generate_agent_from_blueprint` function:

In [ ]:
# Simple generation
# result = await generate_agent_from_blueprint(
#     "blueprints/specs/my_agent.yaml"
# )
#
# # Generate and write
# result = await generate_agent_from_blueprint(
#     "blueprints/specs/my_agent.yaml",
#     output_path="src/agents/my_agent.py",
#     overwrite=True,
# )

print("Convenience function example (commented)")

## Code Generation Modes

AgentBuilder supports two code generation modes:

### 1. Jinja2 Templates (Default)
- Uses external template files in `blueprints/code_templates/`
- More customizable
- Requires template files to be present

### 2. Inline Generator
- Embedded templates, no external files needed
- Self-contained
- Less customizable

In [ ]:
# Use inline generator (no template dependencies)
# result = await builder.run({
#     "blueprint_path": "blueprints/specs/my_agent.yaml",
#     "use_inline_generator": True,
# })

print("Inline generator example (commented)")

## Blueprint Types

### Simple Agents (`type: "simple"`)

Single-message agents with one input/output:

```yaml
blueprint:
  type: "simple"

agent:
  class_name: "MyValidator"
  input:
    type: "string"
  output:
    type: "dict"
  prompts:
    system_prompt: "..."
    user_prompt_template: "..."
```

### LangGraph Workflows (`type: "langgraph"`)

Multi-step workflows with state management:

```yaml
blueprint:
  type: "langgraph"

workflow:
  state_schema:
    content: "str"
    step1_result: "str | None"
    step2_result: "str | None"
    
  steps:
    - name: "step1"
      type: "prompt"  # LLM call
      prompt: "Analyze {content}..."
      
    - name: "step2"
      type: "action"  # Shell command
      action:
        shell:
          command: "git add -A"
          
  edges:
    - from: "step1"
      to: "step2"
    - from: "step2"
      to: "END"
```

## Action Steps (Hybrid Workflows)

Workflows can mix LLM prompts with shell actions:

```yaml
steps:
  - name: "validate_changelog"
    type: "prompt"
    prompt: "Validate and generate commit message..."
    
  - name: "create_branch"
    type: "action"
    action:
      shell:
        command: "git checkout -b release/v{version}"
        timeout: 60
    output_mapping:
      - field: "branch_success"
        from: "success"
      - field: "branch_output"
        from: "output"
```

## Available Blueprints

In [ ]:
from pathlib import Path

# List available blueprints
specs_dir = Path("blueprints/specs")

if specs_dir.exists():
    print("Available Blueprints:")
    print("=" * 50)
    for yaml_file in sorted(specs_dir.glob("*.yaml")):
        print(f"  - {yaml_file.name}")
else:
    print("blueprints/specs directory not found.")
    print("Make sure you're running from the project root.")

## Creating Your Own Blueprint

1. Create a new YAML file in `blueprints/specs/`:

```yaml
# blueprints/specs/my_domain_my_agent.yaml

blueprint:
  version: "1.0"
  name: "my_agent"
  domain: "my_domain"
  description: "Validates X for Y purposes"
  type: "simple"

agent:
  class_name: "MyAgent"
  
  input:
    type: "string"
    description: "Content to validate"
    
  output:
    type: "dict"
    description: "Validation results"
    schema:
      valid: "bool"
      issues: "list[str]"
      summary: "str"
      
  prompts:
    system_prompt: |
      You are an expert validator for X.
      Analyze content and identify issues.
      
    user_prompt_template: |
      Validate this content:
      
      {content}
      
      Return JSON with valid, issues, and summary.
      
  validation_criteria:
    - "Check for A"
    - "Verify B is present"
```

2. Generate the agent:

```python
result = await generate_agent_from_blueprint(
    "blueprints/specs/my_domain_my_agent.yaml",
    output_path="src/agent_workshop/agents/my_domain/my_agent.py",
)
```

## AgentBuilder Pipeline Steps

The AgentBuilder is itself a LangGraph workflow with 4 steps:

```
+----------------+     +--------------------+     +--------------+     +-------------------+
| 1. Load        | --> | 2. Validate        | --> | 3. Generate  | --> | 4. Validate &     |
|    Blueprint   |     |    Blueprint       |     |    Code      |     |    Write          |
+----------------+     +--------------------+     +--------------+     +-------------------+
```

In [ ]:
# View the AgentBuilder graph
graph = builder.build_graph()

print("AgentBuilder Steps:")
for node in graph.nodes:
    if not node.startswith("__"):
        print(f"  - {node}")

## Next Steps

- **[00_getting_started.ipynb](./00_getting_started.ipynb)** - Framework overview
- **[01_deliverable_validator.ipynb](./01_deliverable_validator.ipynb)** - Simple agent example
- **[05_release_pipeline.ipynb](./05_release_pipeline.ipynb)** - Hybrid workflow example